<a href="https://colab.research.google.com/github/koojaekwan/python_practice/blob/master/jaekwankoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.1 준비단계 - 구글드라이브 연동

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


## 1.2 라이브러리 및 데이터 불러오기

In [103]:
import os
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import gc

In [ ]:
print(os.getcwd()) # 현재 경로

/content


In [2]:
data = pd.read_csv("/content/gdrive/My Drive/coding_test/group_call_HMS.csv")

In [ ]:
data

,call_start_day,call_start_time,call_end_day,call_end_time,hashed
0,1,09:14:58.558,1,09:41:30.200,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...
1,17,11:05:05.176,17,13:07:42.515,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...
2,20,02:18:43.172,20,02:28:58.177,86022904c5cf72a54978479c94041f4256d6c3c2a1f71c...
3,22,09:22:01.936,22,09:47:40.192,aafb40d212fe18ff4eafb82fdcf3b53f2161cb3ce59de4...
4,26,06:29:21.182,26,06:50:55.004,c87c2fad141edf323f3787335b54be22945a02fe052448...
...,...,...,...,...,...
6962336,55,08:09:30.086,55,08:29:51.324,a4b6aea6cb58e43911e7cb7d6c0497197db7c4ed16e1c9...
6962337,35,11:25:25.719,35,11:47:41.557,4ec597c569b92bd0e1bae4e2a06e13b9657fb81795e194...
6962338,47,16:29:59.882,47,16:30:27.386,8092e194a750aae539862ed4405f67a6dd5b492e7e57e3...
6962339,48,23:43:52.265,48,23:47:50.285,7176c1516207692857535c30a4650b8e8e586af1fed0fd...


### 1.2.1 데이터 정보

In [4]:
data.isnull().sum()

call_start_day     0
call_start_time    0
call_end_day       0
call_end_time      0
hashed             0
dtype: int64

모든 컬럼에 결측값은 존재하지 않는다.

In [6]:
data.duplicated().sum()

0

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6962341 entries, 0 to 6962340
Data columns (total 5 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   call_start_day   int64 
 1   call_start_time  object
 2   call_end_day     int64 
 3   call_end_time    object
 4   hashed           object
dtypes: int64(2), object(3)
memory usage: 265.6+ MB


In [ ]:
pd.options.display.float_format = '{:.2f}'.format  # 과학적 표기법(Scientific notation)을 사용하지 않는 경우

data.describe()

,call_start_day,call_end_day
count,6962341.00,6962341.00
mean,35.83,35.86
std,16.26,16.25
min,1.00,1.00
25%,23.00,23.00
50%,41.00,41.00
75%,50.00,50.00
max,56.00,57.00


In [ ]:
# temp = pd.Timestamp(data.call_start_time[1])
# print(data.call_start_time[1])
# print(temp)
# print("time : ",temp.time())

## 2.1 통화 & 사람


### 2.1.1 약 100만 rows씩 작업 진행

In [4]:
p = set(np.concatenate(data.loc[0:1000000,"hashed"].str.split(",").values))
p2 = set(np.concatenate(data.loc[1000001:2000000,"hashed"].str.split(",").values))
p3 = set(np.concatenate(data.loc[2000001:3000000,"hashed"].str.split(",").values))
p4 = set(np.concatenate(data.loc[3000001:4000000,"hashed"].str.split(",").values))
p5 = set(np.concatenate(data.loc[4000001:5000000,"hashed"].str.split(",").values))
p6 = set(np.concatenate(data.loc[5000001:6000000,"hashed"].str.split(",").values))
p7 = set(np.concatenate(data.loc[6000001:,"hashed"].str.split(",").values))

한번에 약 700만 개의 행들을 처리하려고 했으나, 램이 부족한 문제로 행들을 구간별로 나눠서 처리하는 방향으로 생각했다.  

### 2.1.2 데이터에서 온전히 통화에 참여한 인원수(중복이 없음)

In [9]:
print("총 통화한 인원수 : ", len(set(list(p)+list(p2)+list(p3)+list(p4)+list(p5)+list(p6)+list(p7))), "명") # 중복이 없는 데이터에서 순수 통화에 참여한 수

총 통화 인원 수 :  268148 명


약 700만 건의 통화리스트에 참여한 사람들 중 unique한 사람들만 카운트한 것이다.

### 2.1.3 데이터에서 1회 통화에 참여한 평균인원 수

In [14]:
data['p_num'] = data['hashed'].str.count(pat = ",") + 1  # 각 통화 건수의 통화한 사람 수

사람들은 각 행별로 , 로 구분되어 있다. 따라서 통화가 일어나는 사건을 나타내는  각 행의 통화에 참여하는 사람수를 구하려면 , 갯수에 1을 더하면 된다.

In [71]:
print("데이터의 총 통화한 사람 : ",data["p_num"].sum(), "명")  # 전체 데이터의 총 통화한 사람 수(사람의 중복이 있음)

데이터의 총 통화한 사람 :  34284282 명


In [17]:
print("1회 통화에 참여한 평균인원수 : ", data["p_num"].sum() / len(data), "명") # 전체 총 통화한 사람수 / 전체 통화 건수 = 1회 통화 평균인원수

1회 통화에 참여한 평균인원수 :  4.924246313129449 명


평균적으로 4.92명이 각 통화에 참여했다.

In [20]:
print("max : ", data["p_num"].max(), ",  min : ", data["p_num"].min())

max :  434 ,  min :  1


혼자 통화에 참여한 사람이 있고, 최대 434명이 함께 통화에 참여한 경우를 확인 가능하다.

### 2.1.4 총 통화시간

총 통화시간은 데이터프레임의 

In [37]:
data["call_end_day"].subtract(data["call_start_day"])

0          0
1          0
2          0
3          0
4          0
          ..
6962336    0
6962337    0
6962338    0
6962339    0
6962340    0
Length: 6962341, dtype: int64

In [44]:
data["call_start_day"].unique()

array([ 1, 17, 20, 22, 26, 36, 40, 47, 50, 54, 15, 24, 42, 43, 48, 49,  8,
       41, 44, 53, 29, 45, 56, 12, 38, 35, 51, 27, 19, 33,  7, 11, 25, 52,
        9, 34, 46,  4, 21, 55, 32, 37,  3, 31, 16,  6, 30, 13, 14, 28, 39,
       23,  2, 10, 18,  5])

In [46]:
data["call_end_day"].unique()

array([ 1, 17, 20, 22, 26, 36, 40, 47, 50, 54, 15, 24, 42, 43, 48, 49,  8,
       41, 44, 53, 29, 46, 56, 12, 38, 35, 51, 27, 16, 19, 33,  7, 11, 25,
       52,  9,  4, 21, 55, 32, 37,  3, 31, 45,  6, 30, 13, 14, 28, 39, 23,
        2, 34, 10, 18,  5, 57])

In [48]:
temp = [x for x in data["hashed"].str.split(",")]

### 2.1.5

In [52]:
len(temp)

6962341

In [70]:
len(data['hashed'].unique())

3184944

통화리스트 중 duplicated로 중복된 데이터가 없음을 확인했다.  
hashed에는 중복된 데이터가 있는 것으로 봐서 몇 그룹들은 한번을 초과하는 전화를 한 것으로 보인다.

### 임시

In [ ]:
# temp = pd.Timestamp(data.call_start_time[1])
# print(data.call_start_time[1])
# print(temp)
# print("time : ",temp.time())

In [27]:
type(data.call_start_time)

pandas.core.series.Series

In [41]:
print(pd.Timestamp("08:09:30.086"))
print(pd.Timestamp("08:29:51.324"))

2020-08-19 08:09:30.086000
2020-08-19 08:29:51.324000


In [45]:
pd.Timestamp("08:29:51.324") - pd.Timestamp("08:09:30.086")

Timedelta('0 days 00:20:21.238000')

In [59]:
data['diff_day'] = data["call_end_day"]-data["call_start_day"]

In [61]:
# data = data.drop(["new_end_day"], axis = 1)

In [63]:
data

,call_start_day,call_start_time,call_end_day,call_end_time,hashed,diff_day
0,1,09:14:58.558,1,09:41:30.200,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...,0
1,17,11:05:05.176,17,13:07:42.515,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...,0
2,20,02:18:43.172,20,02:28:58.177,86022904c5cf72a54978479c94041f4256d6c3c2a1f71c...,0
3,22,09:22:01.936,22,09:47:40.192,aafb40d212fe18ff4eafb82fdcf3b53f2161cb3ce59de4...,0
4,26,06:29:21.182,26,06:50:55.004,c87c2fad141edf323f3787335b54be22945a02fe052448...,0
...,...,...,...,...,...,...
6962336,55,08:09:30.086,55,08:29:51.324,a4b6aea6cb58e43911e7cb7d6c0497197db7c4ed16e1c9...,0
6962337,35,11:25:25.719,35,11:47:41.557,4ec597c569b92bd0e1bae4e2a06e13b9657fb81795e194...,0
6962338,47,16:29:59.882,47,16:30:27.386,8092e194a750aae539862ed4405f67a6dd5b492e7e57e3...,0
6962339,48,23:43:52.265,48,23:47:50.285,7176c1516207692857535c30a4650b8e8e586af1fed0fd...,0


In [117]:
data['call_start_time'] = pd.to_datetime(data['call_start_time'], format='%H:%M:%S.%f', errors='raise')
data['call_end_time'] = pd.to_datetime(data['call_end_time'], format='%H:%M:%S.%f', errors='raise')

In [122]:
# datetime.timedelta(days = data['diff_day'])

TypeError: ignored

In [121]:
data

,call_start_day,call_start_time,call_end_day,call_end_time,hashed,diff_day
0,1,1900-01-01 09:14:58.558,1,1900-01-01 09:41:30.200,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...,0
1,17,1900-01-01 11:05:05.176,17,1900-01-01 13:07:42.515,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...,0
2,20,1900-01-01 02:18:43.172,20,1900-01-01 02:28:58.177,86022904c5cf72a54978479c94041f4256d6c3c2a1f71c...,0
3,22,1900-01-01 09:22:01.936,22,1900-01-01 09:47:40.192,aafb40d212fe18ff4eafb82fdcf3b53f2161cb3ce59de4...,0
4,26,1900-01-01 06:29:21.182,26,1900-01-01 06:50:55.004,c87c2fad141edf323f3787335b54be22945a02fe052448...,0
...,...,...,...,...,...,...
6962336,55,1900-01-01 08:09:30.086,55,1900-01-01 08:29:51.324,a4b6aea6cb58e43911e7cb7d6c0497197db7c4ed16e1c9...,0
6962337,35,1900-01-01 11:25:25.719,35,1900-01-01 11:47:41.557,4ec597c569b92bd0e1bae4e2a06e13b9657fb81795e194...,0
6962338,47,1900-01-01 16:29:59.882,47,1900-01-01 16:30:27.386,8092e194a750aae539862ed4405f67a6dd5b492e7e57e3...,0
6962339,48,1900-01-01 23:43:52.265,48,1900-01-01 23:47:50.285,7176c1516207692857535c30a4650b8e8e586af1fed0fd...,0


In [144]:
data['new_end_time'] = data['call_end_time'] + pd.to_timedelta(data['diff_day'], unit='d')

## 각 통화리스트 별 통화시간

In [160]:
data['call_duration'] = data['new_end_time'] - data['call_start_time']

In [161]:
data

,call_start_day,call_start_time,call_end_day,call_end_time,hashed,diff_day,new_end_time,call_duration
0,1,1900-01-01 09:14:58.558,1,1900-01-01 09:41:30.200,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...,0,1900-01-01 09:41:30.200,00:26:31.642000
1,17,1900-01-01 11:05:05.176,17,1900-01-01 13:07:42.515,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...,0,1900-01-01 13:07:42.515,02:02:37.339000
2,20,1900-01-01 02:18:43.172,20,1900-01-01 02:28:58.177,86022904c5cf72a54978479c94041f4256d6c3c2a1f71c...,0,1900-01-01 02:28:58.177,00:10:15.005000
3,22,1900-01-01 09:22:01.936,22,1900-01-01 09:47:40.192,aafb40d212fe18ff4eafb82fdcf3b53f2161cb3ce59de4...,0,1900-01-01 09:47:40.192,00:25:38.256000
4,26,1900-01-01 06:29:21.182,26,1900-01-01 06:50:55.004,c87c2fad141edf323f3787335b54be22945a02fe052448...,0,1900-01-01 06:50:55.004,00:21:33.822000
...,...,...,...,...,...,...,...,...
6962336,55,1900-01-01 08:09:30.086,55,1900-01-01 08:29:51.324,a4b6aea6cb58e43911e7cb7d6c0497197db7c4ed16e1c9...,0,1900-01-01 08:29:51.324,00:20:21.238000
6962337,35,1900-01-01 11:25:25.719,35,1900-01-01 11:47:41.557,4ec597c569b92bd0e1bae4e2a06e13b9657fb81795e194...,0,1900-01-01 11:47:41.557,00:22:15.838000
6962338,47,1900-01-01 16:29:59.882,47,1900-01-01 16:30:27.386,8092e194a750aae539862ed4405f67a6dd5b492e7e57e3...,0,1900-01-01 16:30:27.386,00:00:27.504000
6962339,48,1900-01-01 23:43:52.265,48,1900-01-01 23:47:50.285,7176c1516207692857535c30a4650b8e8e586af1fed0fd...,0,1900-01-01 23:47:50.285,00:03:58.020000


## 총 통화시간

In [165]:
type(data['call_duration'])

pandas.core.series.Series

In [170]:
data['call_duration'] = pd.to_timedelta(data['call_duration'])

In [172]:
print (data.dtypes)

call_start_day               int64
call_start_time     datetime64[ns]
call_end_day                 int64
call_end_time       datetime64[ns]
hashed                      object
diff_day                     int64
new_end_time        datetime64[ns]
call_duration      timedelta64[ns]
dtype: object


In [234]:
print(data.call_duration[0:5900000].sum())
print(data.call_duration[5900001:].sum())

106067 days 16:21:12.281008
19106 days 01:06:28.455999


600만개의 행이 넘어가면 총 통화시간 계산에 에러가 난다. 따라서 초로 바꾼 뒤, 진행을 해보자

In [260]:
data.call_duration.dt.total_seconds().sum() / (3600*24)

125173.73817542837

약 125173.74일 정도 전체 통화량이 나타내고 있다.